# Exploratory Data Analysis of traffic data at Road 51 Hanasaari

In [15]:
import pandas as pd

Data source: https://www.avoindata.fi/data/fi/dataset/liikennemaarat-helsingissa

Legend: ha=henkilöautot pa=pakettiautot ka=kuorma-autot ra=rekka-autot la=linja-autot mp=moottoripyörät rv=raitiovaunut

In [30]:
data = pd.read_csv('data/hki_liikennemaarat.csv', encoding='latin-1',delimiter=';')
data.head()

,piste,nimi,x_gk25,y_gk25,suunta,aika,vuosi,ha,pa,ka,ra,la,mp,rv,autot
0,A01,LAUTTASAAREN SILTA,25494426,6672169,1.0,0,2011,76,5,1,0,5,0,0,87
1,A01,LAUTTASAAREN SILTA,25494426,6672169,1.0,100,2011,65,5,1,0,4,0,0,75
2,A01,LAUTTASAAREN SILTA,25494426,6672169,1.0,200,2011,61,4,1,0,4,0,0,70
3,A01,LAUTTASAAREN SILTA,25494426,6672169,1.0,300,2011,52,4,1,0,3,0,0,60
4,A01,LAUTTASAAREN SILTA,25494426,6672169,1.0,400,2011,31,2,0,0,2,0,0,35


Remove data about trams and other measurement points.